# Dov2Vec

- An extension of Word2Vec
- Convert a document into a vector representation of a fix-sized numeric values

## TaggedDocument Preparation

In [1]:
import os, gensim
# LEE corpus
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
lee_train_file = test_data_dir + os.sep + 'lee_background.cor'
lee_test_file = test_data_dir + os.sep + 'lee.cor'

print(test_data_dir)
print(lee_train_file)
print(lee_test_file)

/Users/Alvin/opt/anaconda3/envs/python-notes/lib/python3.7/site-packages/gensim/test/test_data
/Users/Alvin/opt/anaconda3/envs/python-notes/lib/python3.7/site-packages/gensim/test/test_data/lee_background.cor
/Users/Alvin/opt/anaconda3/envs/python-notes/lib/python3.7/site-packages/gensim/test/test_data/lee.cor


In [2]:
import smart_open

def read_corpus(file_name, tokens_only=False):
    with smart_open.smart_open(file_name) as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

/Users/Alvin/opt/anaconda3/envs/python-notes/lib/python3.7/site-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


## TaggedDocument Format

- TaggedDocument(words = List(toke, token,...), tags = int())

In [3]:
train_corpus[2]

## A TaggedDocument(List of Word Tokens, Int of Tag)

TaggedDocument(words=['the', 'national', 'road', 'toll', 'for', 'the', 'christmas', 'new', 'year', 'holiday', 'period', 'stands', 'at', 'eight', 'fewer', 'than', 'for', 'the', 'same', 'time', 'last', 'year', 'people', 'have', 'died', 'on', 'new', 'south', 'wales', 'roads', 'with', 'eight', 'fatalities', 'in', 'both', 'queensland', 'and', 'victoria', 'western', 'australia', 'the', 'northern', 'territory', 'and', 'south', 'australia', 'have', 'each', 'recorded', 'three', 'deaths', 'while', 'the', 'act', 'and', 'tasmania', 'remain', 'fatality', 'free'], tags=[2])

## Model Training

In [4]:
%%time
from gensim.models import Doc2Vec
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=100)
model.build_vocab(train_corpus) 
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)

models = [
    # PV-DBOW (Skip-Gram equivalent of Word2Vec)
    Doc2Vec(dm=0, dbow_words=1, vector_size=200, window=8, min_count=10, epochs=50),
    
    # PV-DM w/average (CBOW equivalent of Word2Vec)
    Doc2Vec(dm=1, dm_mean=1, vector_size=200, window=8, min_count=10, epochs =50),
]

/Users/Alvin/.local/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  after removing the cwd from sys.path.


CPU times: user 11.6 s, sys: 741 ms, total: 12.4 s
Wall time: 7.54 s


## Concatenated Model

In [5]:
## Train both PV-DBOW and PV-DM and combine the two

documents = train_corpus
models[0].build_vocab(documents)
models[1].reset_from(models[0])

for model in models:
   model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
new_model = ConcatenatedDoc2Vec((models[0], models[1]))

In [6]:
inferred_vector = model.infer_vector(train_corpus[0].words)
sims = model.docvecs.most_similar([inferred_vector])
print(sims)

[(0, 0.9376226663589478), (255, 0.8408396244049072), (48, 0.8192436695098877), (40, 0.7831258773803711), (272, 0.7820227146148682), (8, 0.7381818294525146), (264, 0.7090743780136108), (33, 0.6963406801223755), (10, 0.676590621471405), (9, 0.6473982334136963)]


:::{note}
A thread on how to use `most_similar()` with `ConcatenatedDoc2Vec`: [link](https://stackoverflow.com/questions/54186233/doc2vec-infer-most-similar-vector-from-concatenateddocvecs)
:::

In [7]:
# model 1
inferred_vector =new_model.models[0].infer_vector(train_corpus[0].words)
sims2 = new_model.models[0].docvecs.most_similar([inferred_vector])
print(sims2)
# model 2
inferred_vector =new_model.models[1].infer_vector(train_corpus[0].words)
sims3 = new_model.models[1].docvecs.most_similar([inferred_vector])
print(sims3)

[(0, 0.946186900138855), (33, 0.47666507959365845), (48, 0.4484424889087677), (40, 0.44048207998275757), (8, 0.37126851081848145), (189, 0.36419862508773804), (264, 0.35121506452560425), (46, 0.3396802544593811), (52, 0.3377082645893097), (222, 0.32457056641578674)]
[(0, 0.9409482479095459), (255, 0.8311561346054077), (48, 0.8207433223724365), (40, 0.7899751663208008), (272, 0.7719666957855225), (8, 0.741024374961853), (264, 0.7003944516181946), (33, 0.6860902309417725), (10, 0.660197377204895), (19, 0.6493285298347473)]


In [8]:
## Doc 1 seems most similar to Doc 255?
print(' '.join(train_corpus[0][0])+'\n')
print(' '.join(train_corpus[255][0])+'\n')
print(' '.join(train_corpus[33][0])+'\n')

hundreds of people have been forced to vacate their homes in the southern highlands of new south wales as strong winds today pushed huge bushfire towards the town of hill top new blaze near goulburn south west of sydney has forced the closure of the hume highway at about pm aedt marked deterioration in the weather as storm cell moved east across the blue mountains forced authorities to make decision to evacuate people from homes in outlying streets at hill top in the new south wales southern highlands an estimated residents have left their homes for nearby mittagong the new south wales rural fire service says the weather conditions which caused the fire to burn in finger formation have now eased and about fire units in and around hill top are optimistic of defending all properties as more than blazes burn on new year eve in new south wales fire crews have been called to new fire at gunning south of goulburn while few details are available at this stage fire authorities says it has clos

In [9]:
## Other vector models 

# # glove

# from gensim.scripts.glove2word2vec import glove2word2vec
# glove_input_file = 'glove.6B.100d.txt'
# word2vec_output_file = 'glove.6B.100d.txt.word2vec'
# glove2word2vec(glove_input_file, word2vec_output_file)

# from gensim.models import KeyedVectors
# filename = 'glove.6B.100d.txt.word2vec'
# model = KeyedVectors.load_word2vec_format(filename, binary=False)

# model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)

# from gensim.models.fasttext import FastText

# ft_model = FastText(size=100)
# ft_model.build_vocab(data)
# model_gensim.train(data, total_examples=ft_model.corpus_count, epochs=ft_model.iter)


# from gensim.models.wrappers.fasttext import FastText

# # Set FastText home to the path to the FastText executable
# ft_home = '/home/bhargav/Gensim/fastText/fasttext'
# # train the model
# model_wrapper = FastText.train(ft_home, train_file)

# print('dog' in model.wv.vocab)
# print('dogs' in model.wv.vocab)

# print('dog' in model)
# print('dogs' in model)

# from gensim.models.wrappers import Wordrank

# wordrank_path = 'wordrank' # path to Wordrank directory
# out_dir = 'model' # name of output directory to save data to
# data = '../../gensim/test/test_data/lee.cor' # sample corpus

# model = Wordrank.train(wordrank_path, data, out_dir, iter=21, dump_period=10)


# varembed_vectors = '../../gensim/test/test_data/varembed_leecorpus_vectors.pkl'
# model = varembed.VarEmbed.load_varembed_format(vectors=varembed_vectors)


# morfessors = '../../gensim/test/test_data/varembed_leecorpus_morfessor.bin'
# model = varembed.VarEmbed.load_varembed_format(vectors=varembed_vectors, morfessor_model=morfessors)

# import os

# poincare_directory = os.path.join(os.getcwd(), 'docs', 'notebooks', 'poincare')
# data_directory = os.path.join(poincare_directory, 'data')
# wordnet_mammal_file = os.path.join(data_directory, 'wordnet_mammal_hypernyms.tsv')

# from gensim.models.poincare import PoincareModel, PoincareKeyedVectors, PoincareRelations
# relations = PoincareRelations(file_path=wordnet_mammal_file, delimiter='\t')
# model = PoincareModel(train_data=relations, size=2, burn_in=0)
# model.train(epochs=1, print_every=500)

# models_directory = os.path.join(poincare_directory, 'models')
# test_model_path = os.path.join(models_directory, 'gensim_model_batch_size_10_burn_in_0_epochs_50_neg_20_dim_50')
# model = PoincareModel.load(test_model_path)